In [ ]:
import pandas as pd


pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

import numpy as np
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler,FunctionTransformer, MinMaxScaler
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import category_encoders as ce
#import shap
from sklearn.feature_selection import SelectKBest, chi2, SelectFromModel
from sklearn import neighbors
from hyperopt import Trials, STATUS_OK, tpe, hp, fmin, STATUS_FAIL, space_eval
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from geopy.distance import geodesic

In [ ]:
class Reduce_memory_usage(TransformerMixin,BaseEstimator):
    def __init__(self):
        pass
    
    def fit(self,X,y=None):
        return self
    
    def transform(self, X, verbose=True):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
            
        numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
        start_mem = X.memory_usage().sum() / 1024 ** 2
        for col in X.columns:
            col_type = X[col].dtypes
            if col_type in numerics:
                c_min = X[col].min()
                c_max = X[col].max()
                if str(col_type)[:3] == "int":
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        X[col] = X[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        X[col] = X[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        X[col] = X[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        X[col] = X[col].astype(np.int64)
                else:
                    if (
                        c_min > np.finfo(np.float16).min
                        and c_max < np.finfo(np.float16).max
                    ):
                        X[col] = X[col].astype(np.float16)
                    elif (
                        c_min > np.finfo(np.float32).min
                        and c_max < np.finfo(np.float32).max
                    ):
                        X[col] = X[col].astype(np.float32)
                    else:
                        X[col] = X[col].astype(np.float64)
        end_mem = X.memory_usage().sum() / 1024 ** 2
        if verbose:
            print(
                "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                    end_mem, 100 * (start_mem - end_mem) / start_mem
                )
            )
        return X

In [ ]:
cl_data = pd.read_parquet('/opt/airflow/datasets/absenteeism/classificacao_1st_especialidade_unidadeprov.parquet')
cl_data.loc[:, 'Label']=np.where(cl_data.Label=='Realizada',0,1)
cl_data.loc[:,'centro_saude']=np.where(cl_data['Tipo Unidade Proveniência'].isin(['U','C']),'1',np.where(cl_data['Tipo Unidade Proveniência'].isna(),'0','2'))

utente = pd.read_parquet('/opt/airflow/datasets/absenteeism/modelacao_1_train_utente_basic_coord.parquet')
cl_data = utente.merge(cl_data, on='NUM_SEQUENCIAL', how='inner') # .drop(columns=['NUM_SEQUENCIAL'])
cl_data.Sexo=cl_data.Sexo.astype('str') 
del utente
cl_data.head(1)

In [ ]:
cl_data = cl_data[cl_data.DataConsulta>='2022-11-30'].reset_index(drop=True)

In [ ]:
conditions = [
    (cl_data.Hora >= 8) & (cl_data.Hora < 10),
    (cl_data.Hora >= 10) & (cl_data.Hora < 12),
    (cl_data.Hora >= 12) & (cl_data.Hora < 14),
    (cl_data.Hora >= 14) & (cl_data.Hora < 17),
    (cl_data.Hora >= 17) & (cl_data.Hora < 19),
    (cl_data.Hora <  8) | (cl_data.Hora >= 19)
    ]
faixas = ["manha1","manha2","almoço","tarde1","tarde2","noite"]
cl_data["day_part"] = np.select(conditions, faixas)

In [ ]:
cl_data.loc[:, 'uni_lat']=np.where(cl_data.Secretariado=='UNIDADE 2',41.106140, np.where(cl_data.Secretariado=='UNIDADE 3 / ESPINHO', 41.002270, None))
cl_data.loc[:, 'uni_long']=np.where(cl_data.Secretariado=='UNIDADE 2',-8.591840, np.where(cl_data.Secretariado=='UNIDADE 3 / ESPINHO', -8.642030, None))

In [ ]:
def distance(row): 
    address1 = (row['utente_lat'], row['utente_lon']) 
    address2 = (row['uni_lat'], row['uni_long']) 
    try:
        if address2[0]==None:
            return np.nan
        else:
            return (geodesic(address1, address2).kilometers) 
    except: 
        return np.nan

In [ ]:
cl_data.loc[:,"dist_btw_uni"] = cl_data.apply(lambda row: distance(row), axis = 1 )

In [ ]:
cl_data.loc[:,"dist_btw"] = np.where(~cl_data.dist_btw_uni.isna(), cl_data.dist_btw_uni, cl_data.dist_btw)

In [ ]:
# 25% feat less importnt
weak_feat = ['Especialidade_CIR. GERAL - POS-OPERATORIA','Especialidade_CIRURGIA DERMATOLOGICA',
 'Especialidade_CIRURGIA DERMATOLOGICA PEDIATRICA','Especialidade_CIRURGIA GERAL',
 'Especialidade_CIRURGIA GERAL-AMBULATORIO UNIDADE3','Especialidade_CIRURGIA PEDIATRICA - ESPINHO',
 'Especialidade_CIRURGIA VASCULAR - EXAMES HEMODINAMICOS', 'Especialidade_CONS. MULT. CANCRO COLO-RETAL',
 'Especialidade_CONS. MULTIDISC. CABECA E PESCOCO', 'Especialidade_CONS. MULTIDISC. PATOL. HEPATOBILIAR',
 'Especialidade_CONS. MULTIDISC. PATOLOGIA GENETICA', 'Especialidade_DERMATOLOGIA - FOTOTERAPIA',
 'Especialidade_DERMATOLOGIA - IMAGIOLOGIA CUTANEA', 'Especialidade_DERMATOLOGIA - ONCOLOGIA CUTANEA',
 'Especialidade_DERMATOLOGIA - RLEC', 'Especialidade_DERMATOLOGIA-ALERGOLOGIA CUTANEA',
 'Especialidade_FOLLOW UP-POS UAVC','Especialidade_FOLLOW UP-POS UAVC CRE',
 'Especialidade_NEUROIMUNOLOGIA', 'Especialidade_NEUROLOGIA - AVC',
 'Especialidade_NEUROLOGIA - CEFALEIAS', 'Especialidade_NEUROLOGIA - DEMENCIAS',
 'Especialidade_NEUROLOGIA - DOENCAS MOVIMENTO', 'Especialidade_NEUROLOGIA - DOENCAS NEUROMUSCULARES',
 'Especialidade_NEUROLOGIA - EPILEPSIA', 'Especialidade_NEUROLOGIA - ESPINHO',
 'Especialidade_NEUROLOGIA - TOXINAS', 'Especialidade_NEUROPEDIATRIA',
 'Especialidade_NEUROPEDIATRIA - ESPINHO', 'Especialidade_OFTALMOLOGIA - GLAUCOMA',
 'Especialidade_OFTALMOLOGIA - IMPLANTO-REFRACTIVA', 'Especialidade_OFTALMOLOGIA - INFLAMACAO OCULAR',
 'Especialidade_OFTALMOLOGIA - NEUROFTALMOLOGIA', 'Especialidade_OFTALMOLOGIA - OCULOPLASTICA/ORBITA',
 'Especialidade_OFTALMOLOGIA - PEDIATRIA/ESTRABISMO', 'Especialidade_OFTALMOLOGIA - RETINA MEDICA',
 'Especialidade_OFTALMOLOGIA - RLEC', 'Especialidade_ORTOPEDIA - COLUNA',
 'Especialidade_ORTOPEDIA - ESPINHO','Especialidade_ORTOPEDIA - JOELHO',
 'Especialidade_ORTOPEDIA - MAO', 'Especialidade_ORTOPEDIA - OMBRO',
 'Especialidade_ORTOPEDIA - PE', 'Especialidade_ORTOPEDIA - RLEC',
 'Especialidade_OTORRINO', 'Especialidade_OTORRINO - ONCOLOGIA',
 'Especialidade_OTORRINO - PEDIATRIA', 'Especialidade_OTORRINO - PRE OPERATORIO',
 'Especialidade_OTORRINO - RINOLOGIA', 'Especialidade_OTORRINO - RLEC',
 'Especialidade_OTORRINO - VOZ', 'Especialidade_PEDIATRIA - ADOLESCENTES',
 'Especialidade_PEDIATRIA - ALERGOLOGIA', 'Especialidade_PEDIATRIA - CDP',
 'Especialidade_PEDIATRIA - CONSULTA VIAJANTE', 'Especialidade_PEDIATRIA - DESENVOLVIMENTO',
 'Especialidade_PEDIATRIA - DESENVOLVIMENTO PEA', 'Especialidade_PEDIATRIA - DESENVOLVIMENTO/ESPINHO',
 'Especialidade_PEDIATRIA - DOENCAS METABOLICAS', 'Especialidade_PEDIATRIA - ENDOCRINOLOGIA/ESPINHO',
 'Especialidade_PEDIATRIA - HEMATOLOGIA', 'Especialidade_PEDIATRIA - INFECCIOLOGIA',
 'Especialidade_PEDIATRIA - NEFROLOGIA', 'Especialidade_PEDIATRIA - PNEUMOLOGIA',
 'Especialidade_PEDIATRIA - REUMATOLOGIA', 'Secretariado_UNIDADE 3 / ESPINHO']

weak_feat = [x.replace('Especialidade_','') for x in weak_feat]

cl_data.loc[cl_data.Especialidade.isin(weak_feat),'Especialidade'] = cl_data[cl_data.Especialidade.isin(weak_feat)].Especialidade.str.split(' - ',expand=True)[0]+'_agg'

cl_data.loc[:,'Especialidade'] = cl_data.Especialidade.str.split(' - ',expand=True)[0]+'_agg'

In [ ]:
assid_count_dia_semana = pd.pivot_table(
    cl_data.groupby(['month_consulta', 'FaixaEtaria', 'dia_consulta', 'Label'], as_index=False)['Sexo'].count(),
    values='Sexo', 
    index=['month_consulta', 'FaixaEtaria', 'dia_consulta'],
    columns=['Label'], aggfunc = np.sum)\
        .reset_index()


if 0 in assid_count_dia_semana.columns.to_list() :
    if 1 in assid_count_dia_semana.columns.to_list() :

        assid_count_dia_semana['assid_dia_semana'] = assid_count_dia_semana[0] /\
            (assid_count_dia_semana[1] + \
            assid_count_dia_semana[0])

    else :
        assid_count_dia_semana['assid_dia_semana'] = 1
    
else :
    assid_count_dia_semana['assid_dia_semana'] = np.nan


In [ ]:
assid_count_day_part = pd.pivot_table(
    cl_data.groupby(['month_consulta', 'FaixaEtaria', 'day_part', 'Label'], as_index=False)['Sexo'].count(),
    values='Sexo', 
    index=['month_consulta', 'FaixaEtaria', 'day_part'],
    columns=['Label'], aggfunc = np.sum)\
        .reset_index()


if 0 in assid_count_day_part.columns.to_list() :
    if 1 in assid_count_day_part.columns.to_list() :

        assid_count_day_part['assid_day_part'] = assid_count_day_part[0] /\
            (assid_count_day_part[1] + \
            assid_count_day_part[0])

    else :
        assid_count_day_part['assid_day_part'] = 1
    
else :
    assid_count_day_part['assid_day_part'] = np.nan


In [ ]:
cl_data = cl_data.merge(assid_count_dia_semana[['month_consulta', 'FaixaEtaria', 'dia_consulta', 'assid_dia_semana']], 
                 on=['month_consulta', 'FaixaEtaria', 'dia_consulta'], how='left') \
        .merge(assid_count_day_part[['month_consulta', 'FaixaEtaria', 'day_part', 'assid_day_part']], 
                 on=['month_consulta', 'FaixaEtaria', 'day_part'], how='left')

del assid_count_dia_semana
del assid_count_day_part

In [ ]:
#"Day_of_week doesn’t have 'x-1' because it goes from 0 to 6. Only variables that start at 1 require the 'x-1'. "

ciclycle=True
if ciclycle:
    cl_data['hora_sin'] = cl_data['Hora'].apply(lambda x: np.sin((x) * (2. * np.pi/31)))
    cl_data['hora_cos'] = cl_data['Hora'].apply(lambda x: np.cos((x) * (2. * np.pi/31)))

    #cl_data["day_of_month"] = cl_data["DataConsulta"].dt.day
    #cl_data['day_of_month_sin'] = cl_data['day_of_month'].apply(lambda x: np.sin((x-1) * (2. * np.pi/31)))
    #cl_data['day_of_month_cos'] = cl_data['day_of_month'].apply(lambda x: np.cos((x-1) * (2. * np.pi/31)))

    #cl_data["day_of_week"] = cl_data["DataConsulta"].dt.dayofweek
    #cl_data['day_of_week_sin'] = cl_data['day_of_week'].apply(lambda x: np.sin((x) * (2. * np.pi/7)))
    #cl_data['day_of_week_cos'] = cl_data['day_of_week'].apply(lambda x: np.cos((x) * (2. * np.pi/7)))

In [ ]:
'Especialidade' in cl_data.columns.tolist()

In [ ]:
X_val = cl_data[cl_data.DataConsulta>=pd.to_datetime('2022-12-01')].drop(columns='Label')
y_val = cl_data[cl_data.DataConsulta>=pd.to_datetime('2022-12-01')].Label

cl_data = X_val

In [ ]:
if ciclycle:
    columns_to_drop = ['Nacionalidade','FaixaEtaria', 'DataConsulta', 'month_consulta','Hora', 'GrupoEspecialidade', #'Especialidade', #
                      'Unidade Proveniência', 'Tipo Unidade Proveniência','dist_btw_uni','uni_lat','uni_long','day_part'] #,'dia_consulta']
else:
    columns_to_drop = ['Nacionalidade','FaixaEtaria', 'DataConsulta', 'month_consulta','GrupoEspecialidade', #'Especialidade', #
                      'Unidade Proveniência', 'Tipo Unidade Proveniência','dist_btw_uni','uni_lat','uni_long','day_part']

if "day_part" in cl_data.columns.tolist():
    columns_to_drop = columns_to_drop + ["Hora"]
    

# test_set = cl_data.loc[:, ~cl_data.columns.isin(columns_to_drop)]

persist_columns = ['Unidade Proveniência', "Secretariado", "DataConsulta", "NUM_SEQUENCIAL", "Especialidade", "GrupoEspecialidade"]

cl_data[persist_columns].head()

In [ ]:
cl_data.shape

In [ ]:
numeric = ['dist_btw', 'Idade', 'Diff_marcacao_consulta', 'assiduidade', 'instituicao', 'num_especialidade', 'median_diff_dias_marcacao_consulta', 'max_diff_dias_marcacao_consulta', 'min_diff_dias_marcacao_consulta', 'primeira_interecao_dias', 'last_interecao_dias', 'last_miss', 'last_realizada', 'sp_assiduidade', 'sp_instituicao', 'sp_num_especialidade', 'sp_median_diff_dias_marcacao_consulta', 'sp_max_diff_dias_marcacao_consulta', 'sp_min_diff_dias_marcacao_consulta', 'sp_primeira_interecao_dias','sp_last_interecao_dias', 'sp_last_miss', 'sp_last_realizada', 'hora_sin', 'hora_cos']
other = ['Sexo', 'TipoAgenda', 'Secretariado', 'Especialidade', 'dia_consulta', 'centro_saude']


#numeric = cl_data.select_dtypes(include= 'number').columns.tolist()
#other = cl_data.select_dtypes(exclude= 'number').columns.tolist()

In [ ]:
test_set = cl_data[other+numeric]

In [ ]:
test_set[numeric]=test_set[numeric].fillna(0)

In [ ]:
import mlflow

logged_model = '/opt/airflow/datasets/models/current/a7f818f9ad574747b3763c95d0274b4d/artifacts/FeatureUnionpipeline/'
import os
assert os.path.exists(logged_model)
feat_model = mlflow.sklearn.load_model(logged_model)

logged_model = '/opt/airflow/datasets/models/current/a7f818f9ad574747b3763c95d0274b4d/artifacts/model/'
import os
assert os.path.exists(logged_model)
model = mlflow.sklearn.load_model(logged_model)

In [ ]:
class get_feature_names(TransformerMixin):
    def transform(self, df):
        return df

    def get_feature_names(self):
        return df.columns.tolist()

validar colunas

In [ ]:
X_trans = feat_model.transform(test_set)
columns_names = feat_model.transformer_list[0][1].named_steps['encode'].get_feature_names() + numeric
X_trans = pd.DataFrame(X_trans, columns=columns_names)

In [ ]:
X_trans.shape

In [ ]:
drop_col = ['TipoAgenda_Sem Agendamento', 'Secretariado_nan', #'Sexo_0', 'centro_saude_0', 
             'Secretariado_CRN', 'dia_consulta_Sunday', 'dia_consulta_Saturday',
             'Secretariado_UNIDADE 3 / ESPINHO','Sexo_1','centro_saude_0','sp_last_miss'] 

In [ ]:
col_to_keep = list(set(X_trans.columns.tolist())-set(drop_col))

In [ ]:
if col_to_keep!=None:
    X_trans = X_trans[col_to_keep]

# Tudo o que está antes é preprocessing #

In [ ]:
test_ds_with_extra = pd.concat([X_trans, cl_data[persist_columns]], axis=1)
# test_ds_with_extra.to_parquet('/absenteeism/test_pre_process_data/classificacaotest_set.parquet', compression='gzip')

In [ ]:
test_ds_with_extra.head()

In [ ]:
pred_probs = model.predict_proba(X_trans)

res = np.zeros(((2, len(pred_probs))))

res[0] = np.amax(pred_probs, axis=1)
res[1] = np.argmax(pred_probs, axis=1)

probs_df = pd.DataFrame(data=np.rot90(res), columns=["Probabilidade", "EstadoPrevisto"])
del res
final = pd.concat([test_ds_with_extra[persist_columns], probs_df], axis=1)
final["EstadoPrevisto"] = np.where(final["EstadoPrevisto"] == 0, 'Realizada', 'Não efetivada e Não Desmarcada')
final.head()